In [54]:
import requests
import os
from dotenv import load_dotenv
import huggingface_hub
from huggingface_hub import InferenceClient
import ast

def get_searches(query):
    load_dotenv()
    API_KEY = os.getenv("API_KEY")
    query = query

    params_related = {"q": query, "api_key": API_KEY, "engine": "google","nums":200}
    related_res = requests.get("https://serpapi.com/search", params=params_related)

    params_auto = {"q": query, "api_key": API_KEY, "engine": "google_autocomplete"}
    auto_res = requests.get("https://serpapi.com/search", params=params_auto)

    related_searches = related_res.json().get("related_searches", []) if related_res.status_code == 200 else []
    autocomplete_suggestions = auto_res.json().get("suggestions", []) if auto_res.status_code == 200 else []

    context=[]
    for i, item in enumerate(related_searches, 1):
        if ((list(item.keys()))[1])=='query':
            context.append(item['query'])

    for i, item in enumerate(autocomplete_suggestions, len(related_searches) + 1):
        context.append(item['value'])
    return check_searches(context, query)


In [ ]:
def check_searches(context, query):
    apikey=os.getenv("apikey")
    repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

    client = InferenceClient(
        api_key=apikey,
        model=repo_id,
        timeout=120,
    )
    query_template = """Given the context: {context}\n
                        is to be filtered based on the given query:{query}. If the context: {context} is not related to {query},\n
                        do not return the context else return the context. Do not give any comments before or after just the context in a new line.
                        The context:{context} is never going to be about movies or songs, its about investments or real life things. Think carefully before returning context: {context}
                        Do not repeat any context all of them should be unique such that no keyword is repeated except for context:{context}
                        Return the output in a single list seperated by commas not a string"""
    query=query 
    prom = query_template.format(query=query, context=context)
    messages = [
        {"role": "system", "content": prom},
        {"role": "user", "content": f"Context: {context}\n\nQuestion: {query}"}
    ] 
    response = client.chat.completions.create(
    model=repo_id,
    messages=messages
    )
    return string_to_list(response['choices'][0]['message']['content'])


In [56]:
def string_to_list(string_input):
    try:
        return ast.literal_eval(string_input)
    except (ValueError, SyntaxError):
        print("Invalid input format")
        return []

In [58]:
a=get_searches('Cars')